# Measuring Discrimination with SolasAI

In [1]:
# In some environments, plotly does not render properly.  If this is the case, run the following code:
# import plotly.io as pio
# pio.renderers.default = "svg"

In [2]:
import solas_disparity as sd

In [3]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

pd.set_option('display.max_columns', 500)

## Importing Data and Building a Model

In [4]:
df = pd.read_csv("hmda.csv.gz", index_col="id")
df.sample(random_state=161803, n=5)

,Low-Priced,Interest Rate,Rate Spread,Loan Amount,Loan-to-Value Ratio,No Intro Rate Period,Intro Rate Period,Property Value,Income,Debt-to-Income Ratio,Term 360,Conforming,State,Product Type,Black,Asian,White,Native American,Hawaiian Or Pacific Islander,Hispanic,Non-Hispanic,Male,Female,Age >= 62,Age < 62,Race,Ethnicity,Sex
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13451,1.0,0.04875,0.00596,155000.0,0.9700,1,0,165000.0,35000.0,0.33,1.0,1.0,FL,conventional,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,White,Hispanic,Male
18248,1.0,0.05750,0.01268,305000.0,1.0000,1,0,295000.0,60000.0,0.55,1.0,1.0,CO,va,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,Unknown
19610,1.0,0.05500,0.01214,485000.0,0.9500,1,0,515000.0,100000.0,0.43,1.0,1.0,CO,conventional,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,1.0,0.0,White,Non-Hispanic,Unknown
3339,1.0,0.03875,-0.00087,675000.0,1.0000,1,0,675000.0,190000.0,0.33,1.0,1.0,VA,va,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,1.0,Black,Non-Hispanic,Unknown
19675,1.0,0.04375,0.00076,275000.0,0.3507,1,0,775000.0,209000.0,0.25,1.0,1.0,AZ,conventional,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,1.0,White,Non-Hispanic,Unknown


In [5]:
features = [
    "Loan Amount",
    "Loan-to-Value Ratio",
    "Intro Rate Period",
    "Property Value",
    "Income",
    "Debt-to-Income Ratio",
    "Term 360",
    "Conforming",
]
label = "Low-Priced"

df['train'] = np.random.choice(a=['train', 'valid'], replace=True, size=len(df), p=[0.8, 0.2])
train = (df['train'] == 'train')


pd.crosstab(df[label], df['train'])

train,train,valid
Low-Priced,,
0.0,1531,383
1.0,14579,3507


In [6]:
params = dict(
    objective="binary:logistic",
    max_depth=3,
    learning_rate=0.02,
    n_estimators=200,
    base_score=df.loc[train, label].mean(),
    random_state=31415,
)
xgb_classifier = xgb.XGBClassifier(**params).fit(X=df.loc[train, features], y=df.loc[train, label])

In [7]:

df.loc[train, 'predictions'] = xgb_classifier.predict_proba(df.loc[train, features])[:, 1]
df.loc[~train, 'predictions'] = xgb_classifier.predict_proba(df.loc[~train, features])[:, 1]


auc_train = metrics.roc_auc_score(y_score=df.loc[train, 'predictions'], y_true=df.loc[train, label])
auc_valid = metrics.roc_auc_score(y_score=df.loc[~train, 'predictions'], y_true=df.loc[~train, label])

print(
    f"\n************************"
    f"\n**** Model ROC-AUC: ****"
    f"\nTraining:          {auc_train:0.3f}"
    f"\nValidation:        {auc_valid:0.3f}"
    f"\nPercent Change:   {auc_valid / auc_train - 1: 0.2%}"
    f"\n************************"
)


************************
**** Model ROC-AUC: ****
Training:          0.865
Validation:        0.856
Percent Change:   -1.02%
************************


In [8]:
df.loc[train, 'predictions'].describe()

cutoff = 0.90

df['Gets Offer'] = (df['predictions'] > cutoff).astype(int)
df['Gets Offer'].value_counts(dropna=False, normalize=True)

1    0.68145
0    0.31855
Name: Gets Offer, dtype: float64

In [9]:
common_info_for_testing = dict(
    group_data=df.loc[~train, :],
    protected_groups=["Black", "Asian", "Native American", "Hispanic", "Female"],
    reference_groups=["White", "White", "White", "Non-Hispanic", "Male"],
    group_categories=["Race", "Race", "Race", "Ethnicity", "Sex"],
)

## Adverse Impact Ratio (AIR)

In [10]:
air = sd.adverse_impact_ratio(
    **common_info_for_testing,
    outcome=df.loc[~train, 'Gets Offer'],
    air_threshold=0.8,
    percent_difference_threshold=0.0,
)

In [11]:
air

## Disparity Calculation: Adverse Impact Ratio

┌───────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ Protected Groups                  │ Black, Asian, Native American, Hispanic, Female                             │
│ Reference Groups                  │ White, White, White, Non-Hispanic, Male                                     │
│ Group Categories                  │ Race, Race, Race, Ethnicity, Sex                                            │
│ AIR Threshold                     │ 0.8                                                                         │
│ % Diff Threshold                  │ 0.0                                                                         │
│ Affected Groups                   │ Black, Native American, Hispanic                                            │
│ Affected Reference                │ White, White, Non-Hispanic                                                  │
│ Affected Categories               │ Race, Race, Ethnicity                                                       │
└───────────────────────────────────┴─────────────────────────────────────────────────────────────────────────────┘

\* Percent Missing: Ethnicity: 14.34%, Race: 13.16%, Sex: 45.71%

## Adverse Impact Ratio Summary Table

Group Category,Group,Reference Group,Observations,Percent Missing,Total,Favorable,Percent Favorable,Percent Difference Favorable,AIR,P-Values,Practically Significant,Shortfall
Race,Black,White,"3,378",13.16%,265.0,109.0,41.13%,26.78%,0.606,0.000,Yes,70.964776
Race,Asian,White,"3,378",13.16%,250.0,214.0,85.60%,-17.69%,1.260,0.000,No,
Race,Native American,White,"3,378",13.16%,21.0,9.0,42.86%,25.05%,0.631,0.019,Yes,5.261360
Race,White,,"3,378",13.16%,"2,839.0","1,928.0",67.91%,,,,,
Ethnicity,Hispanic,Non-Hispanic,"3,332",14.34%,417.0,188.0,45.08%,24.62%,0.647,0.000,Yes,102.684048
Ethnicity,Non-Hispanic,,"3,332",14.34%,"2,915.0","2,032.0",69.71%,,,,,
Sex,Female,Male,"2,112",45.71%,856.0,520.0,60.75%,0.40%,0.993,0.889,No,
Sex,Male,,"2,112",45.71%,"1,256.0",768.0,61.15%,,,,,


## Adverse Impact Ratio by Quantile

In [12]:
airq = sd.adverse_impact_ratio_by_quantile(
    **common_info_for_testing,
    outcome=df.loc[~train, 'predictions'],
    air_threshold=0.8,
    percent_difference_threshold=0.0,
    quantiles=[decile / 10 for decile in range(1, 11)],
    lower_score_favorable=False,
)
airq.plot()

## Standardized Mean Difference (SMD)

In [13]:
smd = sd.standardized_mean_difference(
    **common_info_for_testing,
    outcome=df.loc[~train, 'predictions'],
    smd_threshold=-30,
    lower_score_favorable=False,
)
smd

## Disparity Calculation: SMD

┌───────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ Protected Groups                  │ Black, Asian, Native American, Hispanic, Female                             │
│ Reference Groups                  │ White, White, White, Non-Hispanic, Male                                     │
│ Group Categories                  │ Race, Race, Race, Ethnicity, Sex                                            │
│ SMD Threshold                     │ -30.0                                                                       │
│ Affected Groups                   │ Black, Native American, Hispanic                                            │
│ Affected Reference                │ White, White, Non-Hispanic                                                  │
│ Affected Categories               │ Race, Race, Ethnicity                                                       │
└───────────────────────────────────┴─────────────────────────────────────────────────────────────────────────────┘

\* Percent Missing: Ethnicity: 14.34%, Race: 13.16%, Sex: 45.71%

## SMD Summary Table

Group Category,Group,Reference Group,Observations,Percent Missing,Total,Average Outcome,Std. Dev. of Outcomes,SMD,P-Values,Practically Significant
Race,Black,White,"3,378",13.16%,265.0,0.82,0.11,-77.472,0.000,Yes
Race,Asian,White,"3,378",13.16%,250.0,0.95,0.11,41.198,0.000,No
Race,Native American,White,"3,378",13.16%,21.0,0.85,0.11,-48.232,0.023,Yes
Race,White,,"3,378",13.16%,"2,839.0",0.91,0.11,,,
Ethnicity,Hispanic,Non-Hispanic,"3,332",14.34%,417.0,0.85,0.11,-56.400,0.000,Yes
Ethnicity,Non-Hispanic,,"3,332",14.34%,"2,915.0",0.91,0.11,,,
Sex,Female,Male,"2,112",45.71%,856.0,0.89,0.11,-0.466,0.920,No
Sex,Male,,"2,112",45.71%,"1,256.0",0.89,0.11,,,


## Residual Standardized Mean Difference

In [15]:
rsmd = sd.residual_standardized_mean_difference(
    **common_info_for_testing,
    prediction=df.loc[~train, 'predictions'],
    label=df.loc[~train, label],
    residual_smd_threshold=30,
    lower_score_favorable=True,
)
display(rsmd.plot())
sd.ui.show(rsmd.summary_table)

Group Category,Group,Reference Group,Observations,Percent Missing,Total,Average Prediction,Average Label,Average Residual,Std. Dev. of Residuals,Residual SMD,P-Values,Practically Significant
Race,Black,White,"3,378",13.16%,265.0,0.821731,0.80,-0.025505,0.268311,-10.718782,0.101,No
Race,Asian,White,"3,378",13.16%,250.0,0.949531,0.94,-0.005531,0.268311,-3.274603,0.606,No
Race,Native American,White,"3,378",13.16%,21.0,0.853220,0.76,-0.091316,0.268311,-35.246712,0.101,No
Race,White,,"3,378",13.16%,"2,839.0",0.905163,0.91,0.003255,0.268311,,,
Ethnicity,Hispanic,Non-Hispanic,"3,332",14.34%,417.0,0.847562,0.77,-0.080175,0.268311,-32.988651,0.000,No
Ethnicity,Non-Hispanic,,"3,332",14.34%,"2,915.0",0.908301,0.92,0.008337,0.268311,,,
Sex,Female,Male,"2,112",45.71%,856.0,0.886529,0.89,0.002489,0.268311,2.147863,0.652,No
Sex,Male,,"2,112",45.71%,"1,256.0",0.887032,0.88,-0.003274,0.268311,,,
